When working with data we often do not know the entire data set in advance. For example, think of the dataset of all Google queries, this dataset is constantly changing and becoming larger. This is what leads us to the idea of data streams. These data streams are potentially infinite and non-stationary i.e their distributions are constantly changing.

## Data Stream Model

Let us take a closer look at what data streams look like. We have a system that is receiving input at a rapid rate from one or more input ports (each port being a separate stream). We also think of this input as elements or tuples of data.

Now we want to make decisions or calculations using these data streams. But we run into a few problems, the data is constantly changing and we can not fit all the data in memory, be it secondary or primary. So we need to develop some algorithms to either sample the data or make approximations. 

:::example

Think of Google as the system. They have multiple data streams, one could search queries another maybe from Gmail or Google Drive etc. If we take a look at the search query stream maybe google wants to find out which search queries are popular at the minute to create a trending page.

:::

![dataStreamModel](/img/programming/dataStreamModel.png)

## Sampling a Data Stream

Since we can't store and use the entire data stream we want to take samples of a data stream. However, we want these samples to be fair and represent the entire data stream which can make things very complicated, for example, if we want a sample size of 100 elements we can not just keep track of the last 100 elements as this is not a fair representation of the entire data stream.

When working with samples we are interested in two types of samples either a fixed proportion of the data stream for example 10% of all elements. Or we can get a random sample of fixed size for example 100 elements.

![dataSample](/img/programming/dataSample.png)

### Fixed Proportion Sample

When working with a fixed proportion sample we, for example, want 1 in 10 elements, i.e 10% of the data. However, this does have some issues, if the data stream is not very large then 10% might not be enough data to do our task, be it a calculation, decision or prediction etc. 

You can also imagine the opposite, since the data stream is infinitely large this fixed proportion will grow infinitely which could lead to use not being able to store it in memory.

#### Naive Approach

Let us carry on with the example of Google search queries, we might imagine that Google receives the following tuples in a stream `(user, query, time)`. If we could get a 10% sample of the data stream we can estimate the answer to the question "How often did a user query the same at least twice?".

Our first naive approach might be that we have a 10-sided dice and every time we receive a new tuple we roll the die and if it hits 1 we add the tuple to the sample. If we would be programming this instead of throwing a dice we could generate a random number between 1 and 10, $r \in [1,10]$ and if $r=1$ we store the sample.

However, this approach has a flaw. Imagine each user issues $s$ queries once and $d$ queries twice, the total number of queries executed is $s+2d$. So if we wanted to know the fraction of double queries made it would be $\frac{d}{s+2d}$. However, with this uniform approach our sample will contain $s/10$ of the single queries, $2d/10$ of the duplicate queries at least once but only $d/100$ pairs of duplicates. So our naive approach comes to the solution of $d/(10s+20d)$ which is not correct.

#### Bucket Approach

A different approach would be to take all the queries of 10% of the users. This could be done by defining the user as the key of the tuple and then uniformly hashing the key into 10 buckets. If we then wanted a 20% sample we could just take all the users that were hashed to the buckets with value $h \leq 2$. 

:::todo
Some probabilities would be interesting to see just as in the exercises.
:::

### Fixed Size Sample - Reservoir Sampling

Sometimes we only want to have a fixed sample $S$ of $s$ tuples because maybe we are only able to fit that amount into memory. For this to be a fair and representative sample we want each element to have an equal probability of being in the sample. So if we want a sample size of $s=2$ and at time $t=5$ we have seen 5 elements then we want each element to be in the sample with a probability of $s/t=2/5$.

We can do this using the so-called Reservoir Sampling Algorithm:

1. We store the first $s$ elements of the stream $S$.
2. Then suppose we have seen $t-1$ elements and the $t$-th element arrives. 
   1. With a proability of $s/t$ we keep the $t$-th element. If we keep it we need to make space for it in our sample we do this by kicking out an element picked uniformly at random.
   2. With a probability of $1-s/t$ we discard it

You can simply think of a water reservoir where if fresh water comes in some old water spills/overflows to make space for it.

![reservoirSampling](/img/programming/reservoirSampling.png)

We can prove that this algorithm fits our requirements by using proof by induction:

We assume that after seeing $t$ elements, the sample contains each element seen so far with the probability of $s/t$. For our induction step we need to show that after $t+1$ elements the sample still fulfills the same requirements, i.e the sample contains each element seen so far with the probability of $s/(t+1)$.

Inductive step: For an element already in the sample the probability that it stays in the sample is:

$$
(t+1 \text{ discarded}) + \Big((t+1 \text{ not discarded}) (\text{element not picked})\Big) \newline
= (1- \frac{s}{t+1})+\Big((\frac{s}{t+1})(\frac{s-1}{s})\Big) = \frac{t}{t+1}
$$

So if time is now $t \to t+1$ then the probability that the tuple is still in the sample is 

$$
\frac{s}{t}\cdot \frac{t}{t+1} = \frac{s}{t+1}
$$

## Queries Over a Sliding Window

We very often find ourselves querying a data stream about the most recent input elements. This can be further generalized to processing queries using a sliding window. This sliding window holds $N$ elements. 

![slidingWindow](/img/programming/slidingWindow.png)

We could solve this problem quite easily by just keeping track of the $N$ most recent elements. But what if the $N$ elements take up to much storage and therefore can not be stored in memory? Or we have multiple streams and therefore want to minimize the memory usage as much as possible. 

For this let us move on to our next example of Amazon being our system and that we have for every product $X$ a data stream that consists of 1 or 0 for if that specific item was sold at the time. We could easily come up with the query of how many times has the product $X$ been bought in the last $k$ sales. 

Thanks to our stream structure we can generalize this query to how many 1s are in the last $k$ elements where $k \leq N$.

As mentioned before we don't just want to keep track of the last $N$ elements. Instead, we might think of maintaining 2 counters, $A$ for the number of 1s from the beginning of the stream and $B$ for the number of 0s from the beginning of the stream. If we then assume that the distribution is uniform we would get the following formula:

$$
N \cdot \frac{A}{A+B}
$$

This solution is very simple but not very accurate as the distribution is not necessarily as uniform as we assumed. To prove this we can just think of the sales of Ice cream, it might be sold a lot in summer but much less in winter.

### DGIM Method

The DGIM method is another way to approximate answer the above question of how many 1s are in the last $k$ elements of a data stream. However, it does not assume uniformity and only uses $O(\log^2 N)$ storage which is more than just 2 counters but it makes use of the sliding window instead of looking at the entire data stream. There is however a drawback to this approach, since it is an approximation there will be an error, and in the worst case this method will produce an error of 50%! You might think this is a lot which it is, however, we can reduce this error to a small fraction $> 0$ in exchange for using more memory.

The main idea of the DGIM method is that we split the data stream into blocks with exponentially increasing sizes. Firstly we define that each element has a corresponding timestamp of when it was input into the stream, for example, the first element has the timestamp 1, the next 2 etc.

A block in the DGIM method consists of the following parts:

- The timestamp $t$ of its end. But the be able to represent relevant timestamps to the window and keep storage small we do $t \mod N$. This uses only $\log N$ bits.
- The number of 1s between its beginning and end. This uses $\log(\log N)$ bits because we add the constraint that the number of 1s in a block must be of a power of 2.

We also add a few more constraints to the algorithm apart from the amount of 1s having to be a power of 2:

- There can only be one or two Blocks with the same amount of 1s. 
- Blocks can not overlap.
- Blocks are sorted by size, meaning that "older" blocks can not be smaller than "newer" blocks.
- Blocks disappear when they are out of the window, i.e their end time is larger than $N$.

![dgimBlocks](/img/programming/dgimBlocks.png)

This leads us to have at max $\log N$ blocks, and each block needs $O(\log N)$ bits so we have a total memory usage of $O(log^2 N)$.

#### Updating Blocks

When new values come into the system we need to maintain our data structure. If the new element is 0 we can ignore it but if it is a 1 we need to do the following:

1. Create a new Block of size 1 containing just the new element with the end timestamp being the current time.
2. If there are three blocks of size 1, we need to combine the two oldest blocks into a block of size 2.
3. Recursively check if there are three blocks of the same size and combine the oldest blocks.

![dgimUpdatingBlocks](/img/programming/dgimUpdatingBlocks.png)

#### Querying

Now we just need to know how to query the DGIM data structure we have created to estimate the number of 1s in the sliding window. For this, we sum the sizes, i.e number of 1s of all the blocks apart from the last block. For the last block, we only add half the size. We only take half of the last block because we do not know how many 1s of the last black are actually still within the sliding window. This is also what leads us to our error of 50%. We can prove this by assuming the last block has the size $2^r$, then we assume that half i.e $2^{r-1}$ of its 1s are still in the window. We can also assume that there is at least one block of each size smaller than $2^r$ which sums up to $1+2+4+...+2^{r-1}= 2^r -1$. So our max error can be defined as the following:

$$
\frac{2^{r-1}}{2^r -1}
$$

#### Reducing Error

We can further reduce the error by changing the constraint of having either 1 or 2 blocks of the same size. Instead, we allow there to be either $x-1$ or $x$ blocks of the same size (excluding the largest size which can have between 1 and $x$ blocks). The error is then at most $1/x$. We can tradeoff between the maximum error and the amount of storage used.

:::todo
Example of worst case with error 50%.

This does not make a lot of sense with the end timestamp as it would already be gone as it is larger then $N$
:::

## Filtering a Data Stream

### Bloom Filter

## Counting Distinct Elements

### Flajolet–Martin algorithm

## Moments

### AMS Method

## Counting Frequent Itemsets